# Feature Engineering

This notebook prepares data for model training.

In [1]:
import numpy as np
import pandas as pd

# load pre-processed data
df = pd.read_csv('data/data_cleaned.csv')
df.head()

,Year,Year_Scaled,Year_STD,Month,Keywords,Abstract,Abstract_Cleaned,Abstract Length,Keywords_Cleaned,Number of Keywords,Month_Cleaned
0,2020,1.243352,169.971142,March,"['Capital structure', 'Corporate taxation', 'D...","Absent theoretical guidance, empiricists have ...",absent theoret guidance empiricist forc reli u...,1047,"['capital structure', 'corporate taxation', 'd...",5,3
1,2020,1.243352,169.971142,March,"['Credit spreads', 'LBO risk', 'Structural mod...",Recent decades have witnessed several waves of...,recent decad wit sever wave buyout activity fi...,580,"['credit spreads', 'lbo risk', 'structural mod...",4,3
2,2020,1.243352,169.971142,March,"['Fire sales', 'Liquidity management', 'Mutual...",We develop three novel measures of the incenti...,develop three novel measur incent equiti mutua...,586,"['fire sales', 'liquidity management', 'mutual...",3,3
3,2020,1.243352,169.971142,March,"['Asset pricing', 'Leverage constraints', 'Lot...",We test whether the low-risk effect is driven ...,test whether lowrisk effect driven leverag con...,861,"['asset pricing', 'leverage constraints', 'lot...",5,3
4,2020,1.243352,169.971142,March,"['Gender gap', 'Entrepreneurship', 'Angel inve...",We study whether early stage investors have ge...,studi whether earli stage investor gender bias...,742,"['gender gap', 'entrepreneurship', 'angel inve...",4,3


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from mlxtend.preprocessing import DenseTransformer

# build a data pipeline
pipeline = Pipeline([('vect', CountVectorizer()),                                 # bag-of-words
                     ('lda', LatentDirichletAllocation(n_components=50,           # topic modeling
                                                       random_state=42,
                                                       topic_word_prior=None)),
                     ('tfidf', TfidfTransformer()),
                     ('svd', TruncatedSVD(n_components=10, random_state=42)),     # components reduction
                     ('to_dense', DenseTransformer())])                           # data transform

# build data vector for abstract
x_vector = pipeline.fit_transform(df.Abstract_Cleaned)

# build biased data vector
x_vector_bias = np.zeros((x_vector.shape[0], x_vector.shape[1]+1))
x_vector_bias[:,-1] = df.Year_Scaled / 100
x_vector_bias[:,:-1] = x_vector

# display x_vector_bias
x_vector_bias

array([[ 0.05067446,  0.08134886, -0.0360181 , ..., -0.04204805,
        -0.05446644,  0.01243352],
       [ 0.01195597,  0.02918805,  0.00483241, ...,  0.01169018,
         0.00095615,  0.01243352],
       [ 0.08326695,  0.19427712, -0.16412329, ..., -0.00405454,
         0.17869968,  0.01243352],
       ...,
       [ 0.62136314, -0.15234657,  0.02324684, ...,  0.00152473,
        -0.02040188, -0.02627929],
       [ 0.08388964,  0.17353293, -0.28466068, ..., -0.09554684,
        -0.12300539, -0.02627929],
       [ 0.24049408,  0.21700294, -0.14542568, ...,  0.20482033,
         0.2025117 , -0.02627929]])

In [3]:
# normalize x_vector
x_vector = Normalizer().fit_transform(x_vector)
x_vector

array([[ 0.38622589,  0.6200172 , -0.27451943, ..., -0.1636667 ,
        -0.32047798, -0.41512727],
       [ 0.32305035,  0.78866086,  0.1305716 , ..., -0.03759962,
         0.31586857,  0.02583506],
       [ 0.09677003,  0.22578231, -0.19073855, ...,  0.90290512,
        -0.00471205,  0.20767874],
       ...,
       [ 0.96622139, -0.23689933,  0.0361489 , ..., -0.00857008,
         0.00237096, -0.03172498],
       [ 0.08556933,  0.1770075 , -0.29036031, ...,  0.06048049,
        -0.09745993, -0.12546828],
       [ 0.39128029,  0.35306056, -0.23660542, ..., -0.06404832,
         0.33323964,  0.32948353]])

In [4]:
# the second method
df_x_vector = pd.DataFrame(x_vector, index=None)
df_x_vector = df_x_vector / df_x_vector.std()
x_vector = df_x_vector.to_numpy()

# display x_vector
x_vector

array([[ 1.22002264,  2.2219857 , -1.01177663, ..., -0.73937831,
        -1.06955482, -1.67230182],
       [ 1.02046173,  2.82636214,  0.48123842, ..., -0.16985951,
         1.05417153,  0.10407414],
       [ 0.30568027,  0.80914701, -0.70299143, ...,  4.07895096,
        -0.01572586,  0.83661459],
       ...,
       [ 3.05213089, -0.84898762,  0.13323143, ..., -0.03871609,
         0.00791277, -0.12780114],
       [ 0.27029913,  0.63435036, -1.07016025, ...,  0.27322576,
        -0.32526024, -0.50543736],
       [ 1.23598864,  1.26528023, -0.87203966, ..., -0.28934377,
         1.11214526,  1.32729393]])

### Build Terms Sparse Matrix

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
import json

# create TF-IDF transformer
tfidf = TfidfVectorizer()
tfidf.fit(df.Abstract_Cleaned)
terms_sparse_matrix = tfidf.transform(df.Abstract_Cleaned)
terms_label = tfidf.get_feature_names()

# save data
# to load sparse matrix: sparse_matrix = scipy.sparse.load_npz('/tmp/sparse_matrix.npz')
scipy.sparse.save_npz('data/terms_sparse_matrix.npz', terms_sparse_matrix)

# save term labels
with open("data/terms_label.txt", "w") as fp:
    json.dump(terms_label, fp)

## Save Data for Model Development

In [6]:
# save x_vector
with open('data/x_vector.npy', 'wb') as file:
    np.save(file, x_vector)

In [7]:
# save x_vector_bias
with open('data/x_vector_bias.npy', 'wb') as file:
    np.save(file, x_vector_bias)